In [1]:
import pennylane as qml
from pennylane import numpy as np
import qemzmsepcV2new as qem

In [2]:
n_qubits = 4
dev = qml.device('default.mixed', wires=n_qubits)
nqubitschannel = qem.NqubitsChannel(n_qubits)
pauli_channel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.5)
qemzmsepc = qem.QEMZMSEPC(n_qubits)
p = 0.8

In [3]:
def trotterstep1(rotation_angle_of_rx, rotation_angle_of_rz):
    qml.RX(rotation_angle_of_rx, wires=0)
    qml.RX(rotation_angle_of_rx, wires=1)
    qml.RX(rotation_angle_of_rx, wires=2)
    qml.RX(rotation_angle_of_rx, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.RZ(rotation_angle_of_rz, wires=1)
    qml.RZ(rotation_angle_of_rz, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[1, 2])
    qml.RZ(rotation_angle_of_rz, wires=2)
    qml.CNOT(wires=[1, 2])

In [4]:
rotation_angles = [0, 0.5, 1, 1.5, 2, 2.5, 3]
rotation_angle_of_rz = -0.2
for rotation_angle_of_rx in rotation_angles:
    print(f"rotation_angle_of_rx is {rotation_angle_of_rx}:")
    z_ideal = qml.QNode(qemzmsepc.noise_circuit, dev)(trotterstep1, rotation_angle_of_rx, rotation_angle_of_rz)
    print(f"z_ideal is {z_ideal}")
    
    z_unmitigated = qml.QNode(qemzmsepc.noise_circuit, dev)(trotterstep1, 
                    rotation_angle_of_rx, rotation_angle_of_rz, p=p, 
                    kraus_matrices_of_a_pauli_channel=pauli_channel, need_gate_noise=True,
                    need_measurement_noise=True)
    print(f"z_unmitigated is {z_unmitigated}")
    
    z_mitigated, _  = qemzmsepc.qemzmsepc(trotterstep1, z_unmitigated, p, dev,
                    rotation_angle_of_rx, rotation_angle_of_rz,
                    kraus_matrices_of_a_pauli_channel=pauli_channel)
    print(f"z_mitigated is {z_mitigated}")
    print("------------------")

rotation_angle_of_rx is 0:
z_ideal is 1.0
z_unmitigated is 0.2136913107097822
z_mitigated is 1.000000000000004
------------------
rotation_angle_of_rx is 0.5:
z_ideal is 0.5931327983656769
z_unmitigated is 0.12674732510772257
z_mitigated is 0.5931327983656788
------------------
rotation_angle_of_rx is 1:
z_ideal is 0.08522112911847729
z_unmitigated is 0.018211014781495786
z_mitigated is 0.08522112911848559
------------------
rotation_angle_of_rx is 1.5:
z_ideal is 2.5037531073114627e-05
z_unmitigated is 5.350302831892695e-06
z_mitigated is 2.503753107339751e-05
------------------
rotation_angle_of_rx is 2:
z_ideal is 0.029990685342117292
z_unmitigated is 0.006408748859840944
z_mitigated is 0.02999068534211083
------------------
rotation_angle_of_rx is 2.5:
z_ideal is 0.4119471515970569
z_unmitigated is 0.08802952676793763
z_mitigated is 0.41194715159706286
------------------
rotation_angle_of_rx is 3:
z_ideal is 0.9605668881667448
z_unmitigated is 0.2052647973567689
z_mitigated is 0.96